# Insiders Analysis

Transaction Codes

P: Open market or private purchase of non-derivative or derivative security
M: Exercise or conversion of derivative security exempted puruant to Rule 16b-3. Indicates
   that an insider has exercised options, typically converting them into shares. Can be seen as a form
   of indirect buying, though its a bit more nuanced because the insider may immediately sell the shares 
   they've acquired

S: Indicates an open market or private sale of a non derivative or derivative security.

In [23]:

import pandas as pd
import time
import nasdaqdatalink
import sys

import numpy as np
import sys
sys.path.append('/Users/markwindsor/Desktop/atlas_trade')
from src.utils.technical_indicators import *
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv
NASDAQ_KEY = os.getenv("NASDAQ_KEY")

nasdaqdatalink.ApiConfig.api_key = NASDAQ_KEY

universe_path = "/Users/markwindsor/Desktop/atlas_trade/src/data/universe.csv"
universe_df = pd.read_csv(universe_path)

# Get three month ago date
today = datetime.today()

# Calculate the date three months ago
# This approach considers 30 days in a month for simplicity
three_months_ago = today - timedelta(days=90)

# Format the date in 'YYYY-MM-DD'
formatted_date = three_months_ago.strftime('%Y-%m-%d')




## Screening for clustered Insider buying

In [24]:
transaction_code_map = {
    "P": "Purchase",
    "M": "Exercise of Derivative",
    "S": "Sale"
}

cluster_buying_companies = []

for index, row in universe_df.iterrows():

    insiders = nasdaqdatalink.get_table('SHARADAR/SF2', filingdate={'gte': formatted_date}, ticker=row['ticker'])
    insiders_df = pd.DataFrame(insiders, columns=["ticker", 'filingdate', 'transactiondate', 'formtype', 'issuername', 'ownername', 'officertitle', 'istenpercentowner', 'transactioncode', 'transactionpricepershare', 'transactionvalue', 'securitytitle'])
    insiders_df = insiders_df.sort_values(by="filingdate")

    # Check for any insider purchases in the last 3 months
    insider_buys_df = insiders_df[insiders_df['transactioncode'] == 'P']

    # Here we'll determine cluster insider buying to be 3 or more insiders
    if len(insider_buys_df) > 2:
        clustered_transaction_sum = insider_buys_df['transactionvalue'].sum()
        if clustered_transaction_sum > 1000000:
            row_values = {
                'ticker': row['ticker'],
                'companyName': row['name'],
                'numOfInsiders': len(insider_buys_df),
                'transactionTotal': clustered_transaction_sum
            }
            cluster_buying_companies.append(row_values)

clustered_buying_df = pd.DataFrame(cluster_buying_companies)
clustered_buying_df.to_csv('clusterBuying.csv', index=False)

